In [1]:
"""
--- Day 14: Parabolic Reflector Dish ---

You reach the place where all of the mirrors were pointing: a massive parabolic reflector dish attached to the 
side of another large mountain.

The dish is made up of many small mirrors, but while the mirrors themselves are roughly in the shape of a 
parabolic reflector dish, each individual mirror seems to be pointing in slightly the wrong direction. If the 
dish is meant to focus light, all it's doing right now is sending it in a vague direction.

This system must be what provides the energy for the lava! If you focus the reflector dish, maybe you can go 
where it's pointing and use the light to fix the lava production.

Upon closer inspection, the individual mirrors each appear to be connected via an elaborate system of ropes and 
pulleys to a large metal platform below the dish. The platform is covered in large rocks of various shapes. 
Depending on their position, the weight of the rocks deforms the platform, and the shape of the platform controls 
which ropes move and ultimately the focus of the dish.

In short: if you move the rocks, you can focus the dish. The platform even has a control panel on the side that 
lets you tilt it in one of four directions! The rounded rocks (O) will roll when the platform is tilted, while 
the cube-shaped rocks (#) will stay in place. You note the positions of all of the empty spaces (.) and rocks 
(your puzzle input). For example (A) Start by tilting the lever so all of the rocks will slide north as far as 
they will go (B):
    (A)  O....#....      (B)  OOOO.#.O..
         O.OO#....#           OO..#....#
         .....##...           OO..O##..O
         OO.#O....O           O..#.OO...
         .O.....O#.           ........#.
         O.#..O.#.#           ..#....#.#
         ..O..#O..O           ..O..#.O.O
         .......O..           ..O.......
         #....###..           #....###..
         #OO..#....           #....#....    
You notice that the support beams along the north side of the platform are damaged; to ensure the platform 
doesn't collapse, you should calculate the total load on the north support beams.

The amount of load caused by a single rounded rock (O) is equal to the number of rows from the rock to the south 
edge of the platform, including the row the rock is on. (Cube-shaped rocks (#) don't contribute to load.) So, the 
amount of load caused by each rock in each row is as follows:
    OOOO.#.O.. 10
    OO..#....#  9
    OO..O##..O  8
    O..#.OO...  7
    ........#.  6
    ..#....#.#  5
    ..O..#.O.O  4
    ..O.......  3
    #....###..  2
    #....#....  1
The total load is the sum of the load caused by all of the rounded rocks. In this example, the total load is 136.

Tilt the platform so that the rounded rocks all roll north. 
Afterward, what is the total load on the north support beams?

"""

from typing import Tuple

class Rock:
    def __init__(self, pos: Tuple[int], space: str):
        self.y, self.x = pos
        self.rolling = space == "O"
        self.mark = space
        
    def move(self) -> None:
        if self.rolling:
            self.y -= 1


with open("input.txt") as f:
    layout = f.read().strip()

height = 1 + layout.count("\n")
width = layout.find("\n")

grid = [["."]*width for i in range(height)]
rocks = []

for y, line in enumerate(layout.split("\n")):
    for x, space in enumerate(line):
        if space in "O#":
            rocks.append(Rock((y, x), space))

for rock in rocks:
    while rock.rolling and rock.y-1 >= 0 and grid[rock.y-1][rock.x] == ".":
        rock.move()
    grid[rock.y][rock.x] = rock.mark
    
print(sum((height-i)*line.count("O") for i, line in enumerate(grid)))
    
# total is 109345

109345


In [4]:
"""
--- Part Two ---

The parabolic reflector dish deforms, but not in a way that focuses the beam. To do that, you'll need to move the 
rocks to the edges of the platform. Fortunately, a button on the side of the control panel labeled "spin cycle" 
attempts to do just that!

Each cycle tilts the platform four times so that the rounded rocks roll north, then west, then south, then east. 
After each tilt, the rounded rocks roll as far as they can before the platform tilts in the next direction. After 
one cycle, the platform will have finished rolling the rounded rocks in those four directions in that order.

Here's what happens in the example above after each of the first few cycles:
    After 1 cycle:            After 2 cycles:           After 3 cycles:
    .....#....                .....#....                .....#....
    ....#...O#                ....#...O#                ....#...O#
    ...OO##...                .....##...                .....##...
    .OO#......                ..O#......                ..O#......
    .....OOO#.                .....OOO#.                .....OOO#.
    .O#...O#.#                .O#...O#.#                .O#...O#.#
    ....O#....                ....O#...O                ....O#...O
    ......OOOO                .......OOO                .......OOO
    #...O###..                #..OO###..                #...O###.O
    #..OO#....                #.OOO#...O                #.OOO#...O
This process should work if you leave it running long enough, but you're still worried about the north support 
beams. To make sure they'll survive for a while, you need to calculate the total load on the north support beams 
after 1000000000 cycles.

In the above example, after 1000000000 cycles, the total load on the north support beams is 64.

Run the spin cycle for 1000000000 cycles. Afterward, what is the total load on the north support beams?

"""

import numpy as np


def calculate(grid: np.ndarray) -> int:
    weights = np.array([[n]*grid.shape[1] for n in range(grid.shape[0], 0, -1)])
    return np.sum(np.where(grid == "O", weights, 0))
    
def shift_up(grid: np.ndarray) -> np.ndarray:
    for c in range(grid.shape[1]):
        rocks = np.where(grid[:, c] == "O")[0]
        for r in rocks:
            p = r - 1
            while p >= 0 and grid[p, c] == ".":
                grid[p, c], grid[p + 1, c] = grid[p + 1, c], grid[p, c]
                p -= 1
    return grid


with open("input.txt") as f:
    grid = np.array([list(line.strip()) for line in f])

spin_cycles = 1000000000
visited = {}
visited[grid.tobytes()] = 0

for i in range(1, spin_cycles + 1):
    for j in range(4):
        grid = shift_up(grid)
        grid = np.rot90(grid, k=3)
        
    if seen := visited.get(grid.tobytes(), None):
        period = i - seen
        cycle_pos = (spin_cycles - seen)%period
        billion = seen + cycle_pos
        bytes_arr = list(visited.keys())[billion]
        final = np.frombuffer(bytes_arr, dtype="<U1").reshape(grid.shape)
        break
    visited[grid.tobytes()] = i
    
print(calculate(final))

# total load is 112452

112452
